# COVID Cough Capstone - Pre-Processing 

#### Goals of this notebook: 

* Creating dummy features 
    
* Scale standardization
* Split data into training and testing subsets

In [1]:
import os, json 
import pandas as pd
import numpy as np
import ffmpeg 
from datetime import datetime
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt

In [2]:
path_to_files = '/Users/yevheniyashevchenko/Documents - Yevheniya’s MacBook Pro/COVIDCoughCapstone/public_dataset/'
json_files = [loc_json for loc_json in os.listdir(path_to_files) if loc_json.endswith('.json')]

### Loading the audio files 

In [3]:
fileExt = ['.webm', '.ogg']
webm_files = [files for files in os.listdir(path_to_files) if files.endswith(tuple(fileExt))]

In [4]:
audios = []

for file in webm_files: 
    with open(path_to_files + file) as sound_files:
        audios.append(sound_files)

In [5]:
print(audios[0])

<_io.TextIOWrapper name='/Users/yevheniyashevchenko/Documents - Yevheniya’s MacBook Pro/COVIDCoughCapstone/public_dataset/cf593695-ce85-4a56-81c6-14617de66f71.ogg' mode='r' encoding='UTF-8'>


In [6]:
import random
import librosa

In [7]:
import warnings 
warnings.filterwarnings(action='once')

In [8]:
audios_sample = random.sample(audios, 10)

In [10]:
audio_list = []
warnings.filterwarnings('ignore')
for i,s in enumerate(audios, start=1):
    audio_dict = {}
    with open(path_to_files + webm_files[i],"rb") as s_:
        y, sr = librosa.load(path_to_files + webm_files[i], sr=44100, mono=False)
        rms_1 = librosa.feature.rms(y=y)
        spec_cent_1 = librosa.feature.spectral_centroid(y=y, sr=sr)
        spec_bw_1 = librosa.feature.spectral_bandwidth(y=y, sr=sr)
        rolloff_1 = librosa.feature.spectral_rolloff(y=y, sr=sr)
        zcr_1 = librosa.feature.zero_crossing_rate(y)
        mfcc_1 = librosa.feature.mfcc(y=y, sr=sr)
        audio_dict['rms_mean'] = np.mean(rms_1)
        audio_dict['rms_max'] = np.max(rms_1)
        audio_dict['spec_cent_mean'] = np.mean(spec_cent_1)
        audio_dict['spec_cent_max'] = np.max(spec_cent_1)
        audio_dict['spec_bw_mean'] = np.mean(spec_bw_1)
        audio_dict['spec_bw_max'] = np.max(spec_bw_1)
        audio_dict['rolloff_mean'] = np.mean(rolloff_1)
        audio_dict['rolloff_max'] = np.max(rolloff_1)
        audio_dict['zcr_mean'] = np.mean(zcr_1)
        audio_dict['zcr_max'] = np.max(zcr_1)
        #audio_dict['mfcc_min_max'] = 
        #audio_dict['mfcc_mean'] = []
        for e in mfcc_1: 
            means_e = [np.mean(e)]
        audio_dict['mfcc_mean']= np.mean(e)
        #for e in mfcc_1: 
        # for e in mfcc_1: 
        #     audio_dict['local_min_max'] = [min(e), max(e)]
        loc_max = [np.max(e) for e in mfcc_1]
        audio_dict['mfcc_max'] = np.max(loc_max)
        loc_min = [np.min(e) for e in mfcc_1]
        audio_dict['mfcc_min'] = np.min(loc_max)
        # local_min_max = [[min(e), max(e)] for e in mfcc_1]
        # min_y = 412413131124141
        # max_y = -341243141341411341
        # for y in local_min_max:
        #     if y[0] < min_y: min_y = y[0]
        #     if y[1] < min_y: min_y = y[1]
        #     if y[0] > max_y: max_y = y[0]
        #     if y[1] > max_y: max_y = y[1]
        # audio_dict['mfcc_min'] = min(local_min_max)
        # audio_dict['mfcc_max'] = max(local_min_max)
       # audio_dict['mfcc'] = mfcc_1
    audio_list.append(audio_dict)   
#I need to note what is the index of the sample datasets that I`m taking 


IndexError: list index out of range

In [11]:
df_audio_features = pd.DataFrame(audio_list)

In [12]:
df_audio_features.head()

,rms_mean,rms_max,spec_cent_mean,spec_cent_max,spec_bw_mean,spec_bw_max,rolloff_mean,rolloff_max,zcr_mean,zcr_max,mfcc_mean,mfcc_max,mfcc_min
0,0.003272,0.045307,2725.790917,8643.121363,2528.608396,5737.569994,4741.798134,15999.169922,0.093798,0.274414,-3.664643,254.662933,-277.932037
1,0.040019,0.268396,3056.461030,18915.416645,3377.612557,6350.198655,5619.722946,21317.871094,0.073610,0.343750,-4.290027,214.227386,-11.241102
2,0.068667,0.455073,2532.270462,10120.249909,2460.946615,6109.350601,4337.534999,17205.029297,0.105732,0.428711,-6.005171,248.925339,-88.177902
3,0.037967,0.444520,2673.069302,10120.249909,2734.758435,6604.815393,5077.971004,17205.029297,0.081322,0.653809,-4.017477,227.154419,-91.822929
4,0.034374,0.378970,4275.925307,10153.549088,4492.883108,6687.462198,9315.142500,16946.630859,0.132605,0.365723,-2.786305,215.575592,-22.162752


In [13]:
len(df_audio_features)

27549

## DO NOT RERUN
### IndexError: list index out of range because we skipped the first entry 

### Loading Participant Information

In [50]:
output_list = []
for i,file in enumerate(json_files):
    if i == 0: continue
    data_dict = {}
    with open(path_to_files + file,"r") as json_open:
        data = json.loads(json_open.read())
        #print(data)
        for k,v in data.items(): 
            data_dict[k]= v
    output_list.append(data_dict)


In [51]:
df = pd.DataFrame(output_list)

In [52]:
len(df)

27549

In [53]:
df['datetime'] = pd.to_datetime(df['datetime'])

In [54]:
expert_columns = [x for x in df.columns if 'expert_' in x]

In [55]:
def extract_keys(x):
    if isinstance(x, dict):
        return list(x.keys())
    return []

In [56]:
def create_row_expert_dict(x, col_dict,col_name):
    if isinstance(x, dict):
        output = {}
        for key in list(col_dict.keys()):
            if key in x:
                output[col_name+'_'+key] = x[key]
        return output
    return col_dict

In [57]:
for col in expert_columns:
    col_extracted_keys = df[col].apply(lambda x: extract_keys(x))
    col_keys = []
    for row in col_extracted_keys: col_keys = list(np.unique(col_keys + row))
    col_dict = {x: False for x in col_keys}
    df_expert = pd.DataFrame(df[col].apply(lambda x: create_row_expert_dict(x, col_dict,col)).to_dict()).T
    df = pd.concat([df,df_expert], axis=1)

In [58]:
for k in col_keys:
    df = df.drop([k], axis=1)

In [59]:
df.head()

,datetime,cough_detected,latitude,longitude,age,gender,respiratory_condition,fever_muscle_pain,status,expert_labels_1,...,expert_labels_4_choking,expert_labels_4_congestion,expert_labels_4_cough_type,expert_labels_4_diagnosis,expert_labels_4_dyspnea,expert_labels_4_nothing,expert_labels_4_quality,expert_labels_4_severity,expert_labels_4_stridor,expert_labels_4_wheezing
0,2020-04-11 19:37:13.279037+00:00,0.0396,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-11-27 12:08:05.795269+00:00,0.8014,48.3,-1.5,67,female,False,False,healthy,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-04-14 06:45:05.925650+00:00,0.6086,39.8,64.4,22,male,False,False,healthy,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-04-19 07:58:56.194180+00:00,0.0977,13.7,-89.2,20,male,True,False,healthy,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-04-11 02:01:08.084013+00:00,0.0535,NaN,NaN,36,female,False,False,COVID-19,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [60]:
#Leave NAN and changed the values to floats
df['age'] = pd.to_numeric(df['age'], errors='coerce')
df['cough_detected'] = df['cough_detected'].astype(float)

In [61]:
df = df.drop(['expert_labels_1', 'expert_labels_2', 'expert_labels_3', 'expert_labels_4'], axis=1)

In [62]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27549 entries, 0 to 27548
Data columns (total 49 columns):
 #   Column                      Non-Null Count  Dtype              
---  ------                      --------------  -----              
 0   datetime                    27549 non-null  datetime64[ns, UTC]
 1   cough_detected              27549 non-null  float64            
 2   latitude                    16083 non-null  object             
 3   longitude                   16083 non-null  object             
 4   age                         15218 non-null  float64            
 5   gender                      16224 non-null  object             
 6   respiratory_condition       16224 non-null  object             
 7   fever_muscle_pain           16224 non-null  object             
 8   status                      16224 non-null  object             
 9   expert_labels_1_choking     820 non-null    object             
 10  expert_labels_1_congestion  820 non-null    object        

In [63]:
categorical = [] 
for col in df.columns[5:]:
    categorical.append(col)
#Trying to collect all the column names 


In [64]:
print(categorical)

['gender', 'respiratory_condition', 'fever_muscle_pain', 'status', 'expert_labels_1_choking', 'expert_labels_1_congestion', 'expert_labels_1_cough_type', 'expert_labels_1_diagnosis', 'expert_labels_1_dyspnea', 'expert_labels_1_nothing', 'expert_labels_1_quality', 'expert_labels_1_severity', 'expert_labels_1_stridor', 'expert_labels_1_wheezing', 'expert_labels_2_choking', 'expert_labels_2_congestion', 'expert_labels_2_cough_type', 'expert_labels_2_diagnosis', 'expert_labels_2_dyspnea', 'expert_labels_2_nothing', 'expert_labels_2_quality', 'expert_labels_2_severity', 'expert_labels_2_stridor', 'expert_labels_2_wheezing', 'expert_labels_3_choking', 'expert_labels_3_congestion', 'expert_labels_3_cough_type', 'expert_labels_3_diagnosis', 'expert_labels_3_dyspnea', 'expert_labels_3_nothing', 'expert_labels_3_quality', 'expert_labels_3_severity', 'expert_labels_3_stridor', 'expert_labels_3_wheezing', 'expert_labels_4_choking', 'expert_labels_4_congestion', 'expert_labels_4_cough_type', 'exper

In [65]:
df = pd.get_dummies(df, columns=categorical,drop_first=True)
df.head()

,datetime,cough_detected,latitude,longitude,age,gender_male,gender_other,respiratory_condition_True,fever_muscle_pain_True,status_healthy,...,expert_labels_4_dyspnea_True,expert_labels_4_nothing_True,expert_labels_4_quality_no_cough,expert_labels_4_quality_ok,expert_labels_4_quality_poor,expert_labels_4_severity_pseudocough,expert_labels_4_severity_severe,expert_labels_4_severity_unknown,expert_labels_4_stridor_True,expert_labels_4_wheezing_True
0,2020-04-11 19:37:13.279037+00:00,0.0396,NaN,NaN,NaN,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2020-11-27 12:08:05.795269+00:00,0.8014,48.3,-1.5,67.0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,2020-04-14 06:45:05.925650+00:00,0.6086,39.8,64.4,22.0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,2020-04-19 07:58:56.194180+00:00,0.0977,13.7,-89.2,20.0,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,2020-04-11 02:01:08.084013+00:00,0.0535,NaN,NaN,36.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [66]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27549 entries, 0 to 27548
Data columns (total 80 columns):
 #   Column                                         Non-Null Count  Dtype              
---  ------                                         --------------  -----              
 0   datetime                                       27549 non-null  datetime64[ns, UTC]
 1   cough_detected                                 27549 non-null  float64            
 2   latitude                                       16083 non-null  object             
 3   longitude                                      16083 non-null  object             
 4   age                                            15218 non-null  float64            
 5   gender_male                                    27549 non-null  uint8              
 6   gender_other                                   27549 non-null  uint8              
 7   respiratory_condition_True                     27549 non-null  uint8              
 8   fever_

In [76]:
df_audio_features.head()

,rms_mean,rms_max,spec_cent_mean,spec_cent_max,spec_bw_mean,spec_bw_max,rolloff_mean,rolloff_max,zcr_mean,zcr_max,mfcc_mean,mfcc_max,mfcc_min
0,0.003272,0.045307,2725.790917,8643.121363,2528.608396,5737.569994,4741.798134,15999.169922,0.093798,0.274414,-3.664643,254.662933,-277.932037
1,0.040019,0.268396,3056.461030,18915.416645,3377.612557,6350.198655,5619.722946,21317.871094,0.073610,0.343750,-4.290027,214.227386,-11.241102
2,0.068667,0.455073,2532.270462,10120.249909,2460.946615,6109.350601,4337.534999,17205.029297,0.105732,0.428711,-6.005171,248.925339,-88.177902
3,0.037967,0.444520,2673.069302,10120.249909,2734.758435,6604.815393,5077.971004,17205.029297,0.081322,0.653809,-4.017477,227.154419,-91.822929
4,0.034374,0.378970,4275.925307,10153.549088,4492.883108,6687.462198,9315.142500,16946.630859,0.132605,0.365723,-2.786305,215.575592,-22.162752


In [83]:
df_audio = df.join(df_audio_features)

In [84]:
df_audio.head()

,datetime,cough_detected,latitude,longitude,age,gender_male,gender_other,respiratory_condition_True,fever_muscle_pain_True,status_healthy,...,spec_cent_max,spec_bw_mean,spec_bw_max,rolloff_mean,rolloff_max,zcr_mean,zcr_max,mfcc_mean,mfcc_max,mfcc_min
0,2020-04-11 19:37:13.279037+00:00,0.0396,NaN,NaN,NaN,0,0,0,0,0,...,8643.121363,2528.608396,5737.569994,4741.798134,15999.169922,0.093798,0.274414,-3.664643,254.662933,-277.932037
1,2020-11-27 12:08:05.795269+00:00,0.8014,48.3,-1.5,67.0,0,0,0,0,1,...,18915.416645,3377.612557,6350.198655,5619.722946,21317.871094,0.073610,0.343750,-4.290027,214.227386,-11.241102
2,2020-04-14 06:45:05.925650+00:00,0.6086,39.8,64.4,22.0,1,0,0,0,1,...,10120.249909,2460.946615,6109.350601,4337.534999,17205.029297,0.105732,0.428711,-6.005171,248.925339,-88.177902
3,2020-04-19 07:58:56.194180+00:00,0.0977,13.7,-89.2,20.0,1,0,1,0,1,...,10120.249909,2734.758435,6604.815393,5077.971004,17205.029297,0.081322,0.653809,-4.017477,227.154419,-91.822929
4,2020-04-11 02:01:08.084013+00:00,0.0535,NaN,NaN,36.0,0,0,0,0,0,...,10153.549088,4492.883108,6687.462198,9315.142500,16946.630859,0.132605,0.365723,-2.786305,215.575592,-22.162752


In [85]:
df_audio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27549 entries, 0 to 27548
Data columns (total 93 columns):
 #   Column                                         Non-Null Count  Dtype              
---  ------                                         --------------  -----              
 0   datetime                                       27549 non-null  datetime64[ns, UTC]
 1   cough_detected                                 27549 non-null  float64            
 2   latitude                                       16083 non-null  object             
 3   longitude                                      16083 non-null  object             
 4   age                                            15218 non-null  float64            
 5   gender_male                                    27549 non-null  uint8              
 6   gender_other                                   27549 non-null  uint8              
 7   respiratory_condition_True                     27549 non-null  uint8              
 8   fever_

In [67]:
from sklearn import preprocessing
from sklearn.preprocessing import minmax_scale
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [68]:
#Features are in X and target are in y

y_var = []
for col in df.columns[7:]:
    y_var.append(col)


In [99]:
# y [patient had covid or not - expert labels & participant information]
y = df_audio[y_var]

In [100]:
# X Sound features
X = df_audio.iloc[:,80:92]

In [101]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42) 